In [1]:
import os
from datetime import datetime
import cv2
from fpdf import FPDF
import re

# 指定视频文件所在的目录路径 (绝对路径)
directory_path = r'D:\download\gene\ppt_real'

# 指定输出图像和PDF的目录 (绝对路径)
output_directory = os.path.join(directory_path, 'ppt')
os.makedirs(output_directory, exist_ok=True)  # 如果目录不存在，则创建

# 获取目录下所有文件的列表
file_list = os.listdir(directory_path)

# 按照文件的修改时间进行排序
file_list.sort(key=lambda x: os.path.getmtime(os.path.join(directory_path, x)))

# 遍历排序后的文件列表
for file_name in file_list:
    file_path = os.path.join(directory_path, file_name)

    # 检查是否为文件而非子目录
    if not os.path.isfile(file_path):
        continue  # 跳过非文件项

    # 获取文件的修改时间
    modified_time = datetime.fromtimestamp(os.path.getmtime(file_path))
    print(f"正在处理文件: {file_path}")
    print(f"文件名：{file_name}，修改时间：{modified_time}")

    # 打开视频文件
    cap = cv2.VideoCapture(file_path)

    # 检查视频是否成功打开
    if not cap.isOpened():
        print(f"无法打开视频文件: {file_path}")
        continue  # 跳过无法打开的视频文件

    # 初始化变量
    prev_frame_gray_1 = None
    prev_frame_gray_2 = None
    page_images = []
    i = 0

    while True:
        ret, frame = cap.read()

        if not ret:
            break  # 读取完毕或发生错误

        # 检查帧是否为空
        if frame is None:
            continue

        # 确保感兴趣区域的坐标在帧的边界内
        if frame.shape[0] < 720 or frame.shape[1] < 1280:
            print(f"帧尺寸不足，跳过该帧，帧尺寸：{frame.shape}")
            continue

        # 调整感兴趣区域的坐标以适应帧的实际尺寸
        roi_1 = frame[480:660, 1060:1200]  # 感兴趣区域1，往上移动100像素
        roi_2 = frame[280:500, 340:540]    # 感兴趣区域2，扩大一些

        # 将感兴趣区域转换为灰度图像
        roi_gray_1 = cv2.cvtColor(roi_1, cv2.COLOR_BGR2GRAY)
        roi_gray_2 = cv2.cvtColor(roi_2, cv2.COLOR_BGR2GRAY)

        # 初始化前一帧灰度图像
        if prev_frame_gray_1 is None:
            prev_frame_gray_1 = roi_gray_1.copy()
            prev_frame_gray_2 = roi_gray_2.copy()
            continue  # 跳过第一帧的差分检测

        # 计算当前帧和前一帧的差分图像
        frame_diff_1 = cv2.absdiff(roi_gray_1, prev_frame_gray_1)
        frame_diff_2 = cv2.absdiff(roi_gray_2, prev_frame_gray_2)

        # 对差分图像进行阈值处理
        _, threshold_1 = cv2.threshold(frame_diff_1, 30, 255, cv2.THRESH_BINARY)
        _, threshold_2 = cv2.threshold(frame_diff_2, 30, 255, cv2.THRESH_BINARY)

        # 统计阈值图像中非零像素的数量
        diff_pixels_1 = cv2.countNonZero(threshold_1)
        diff_pixels_2 = cv2.countNonZero(threshold_2)

        # 如果两个区域的差分像素数量都超过阈值，则认为帧发生了变化
        if (diff_pixels_1 > 100 and diff_pixels_2 > 300) or diff_pixels_1 > 150 or diff_pixels_2 > 1000:
            print(f"帧发生变化: {i + 1}, 差分像素1: {diff_pixels_1}, 差分像素2: {diff_pixels_2}")

            # 提取文件名中的编号部分
            match = re.search(r'第(\d+)讲', file_name)
            if match:
                lecture_number = match.group(1)
            else:
                lecture_number = 'unknown'

            # 生成简化的文件名
            simplified_file_name = f"{lecture_number}_{i + 1}.jpg"
            page_path = os.path.join(output_directory, simplified_file_name)

            # 尝试保存图像
            try:
                # 检查输出目录是否存在
                if not os.path.exists(output_directory):
                    os.makedirs(output_directory, exist_ok=True)
                    print(f"创建输出目录: {output_directory}")

                # 保存图像
                success = cv2.imwrite(page_path, frame)
                if success:
                    print(f"图像已保存: {page_path}")
                    page_images.append(page_path)
                else:
                    print(f"图像保存失败: {page_path}")
            except Exception as e:
                print(f"保存图像时发生错误: {str(e)}")
            i += 1

        # 更新前一帧
        prev_frame_gray_1 = roi_gray_1.copy()
        prev_frame_gray_2 = roi_gray_2.copy()

        # 显示当前帧 (可选，运行时可关闭窗口加快速度)
        cv2.imshow("Frame", frame)

        # 跳过一定数量的帧以加快处理速度
        current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
        max_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # 计算目标帧的索引（例如跳500帧）
        target_frame = current_frame + 100
        if target_frame >= max_frames:
            break  # 超过总帧数，结束处理
        cap.set(cv2.CAP_PROP_POS_FRAMES, target_frame)

        # 按下 'q' 键退出循环 (可选，已注释)
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break

    cap.release()
    cv2.destroyAllWindows()

    # 创建PDF文件
    pdf = FPDF(orientation='L', unit='mm', format='A4')  # 使用横向A4纸
    pdf.set_auto_page_break(auto=True, margin=15)

    for image_path in page_images:
        if os.path.exists(image_path):
            pdf.add_page()
            # 计算图片的宽度和高度，以适应PDF页面
            pdf_width = pdf.w   # 左右各5的边距
            pdf_height = pdf.h - 20  # 上下各10的边距
            pdf.image(image_path, x=5, y=10, w=pdf_width, h=pdf_height)
        else:
            print(f"图像文件不存在: {image_path}")

    # 保存PDF
    pdf_file_name = os.path.splitext(file_name)[0] + '.pdf'
    pdf_path = os.path.join(output_directory, pdf_file_name)
    try:
        pdf.output(pdf_path)
        print(f"已保存PDF: {pdf_path}")
    except Exception as e:
        print(f"保存PDF时发生错误: {str(e)}")

正在处理文件: D:\download\gene\ppt_real\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第2讲)_1.mp4
文件名：动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第2讲)_1.mp4，修改时间：2025-01-06 21:10:33.576509
帧发生变化: 1, 差分像素1: 111, 差分像素2: 11944
图像已保存: D:\download\gene\ppt_real\ppt\2_1.jpg
帧发生变化: 2, 差分像素1: 111, 差分像素2: 9618
图像已保存: D:\download\gene\ppt_real\ppt\2_2.jpg
帧发生变化: 3, 差分像素1: 0, 差分像素2: 2899
图像已保存: D:\download\gene\ppt_real\ppt\2_3.jpg
帧发生变化: 4, 差分像素1: 0, 差分像素2: 9462
图像已保存: D:\download\gene\ppt_real\ppt\2_4.jpg
帧发生变化: 5, 差分像素1: 733, 差分像素2: 13995
图像已保存: D:\download\gene\ppt_real\ppt\2_5.jpg
帧发生变化: 6, 差分像素1: 0, 差分像素2: 1342
图像已保存: D:\download\gene\ppt_real\ppt\2_6.jpg
帧发生变化: 7, 差分像素1: 0, 差分像素2: 1341
图像已保存: D:\download\gene\ppt_real\ppt\2_7.jpg
帧发生变化: 8, 差分像素1: 732, 差分像素2: 8688
图像已保存: D:\download\gene\ppt_real\ppt\2_8.jpg
帧发生变化: 9, 差分像素1: 17553, 差分像素2: 0
图像已保存: D:\download\gene\ppt_real\ppt\2_9.jpg
帧发生变化: 10, 差分像素1: 17279, 差分像素2: 4818
图像已保存: D:\download\gene\ppt_real\ppt\2_10.jpg
帧发生变化: 11, 差分像素1: 2535, 差分像素2: 13382
图像已保存: D:\download\gene\pp

帧发生变化: 20, 差分像素1: 4116, 差分像素2: 43106
图像已保存: D:\download\gene\ppt_real\ppt\1_20.jpg
帧发生变化: 21, 差分像素1: 0, 差分像素2: 39476
图像已保存: D:\download\gene\ppt_real\ppt\1_21.jpg
帧发生变化: 22, 差分像素1: 0, 差分像素2: 8278
图像已保存: D:\download\gene\ppt_real\ppt\1_22.jpg
帧发生变化: 23, 差分像素1: 0, 差分像素2: 2452
图像已保存: D:\download\gene\ppt_real\ppt\1_23.jpg
帧发生变化: 24, 差分像素1: 78, 差分像素2: 11528
图像已保存: D:\download\gene\ppt_real\ppt\1_24.jpg
帧发生变化: 25, 差分像素1: 78, 差分像素2: 9486
图像已保存: D:\download\gene\ppt_real\ppt\1_25.jpg
帧发生变化: 26, 差分像素1: 0, 差分像素2: 5842
图像已保存: D:\download\gene\ppt_real\ppt\1_26.jpg
帧发生变化: 27, 差分像素1: 0, 差分像素2: 6184
图像已保存: D:\download\gene\ppt_real\ppt\1_27.jpg
帧发生变化: 28, 差分像素1: 1464, 差分像素2: 4821
图像已保存: D:\download\gene\ppt_real\ppt\1_28.jpg
帧发生变化: 29, 差分像素1: 2463, 差分像素2: 23837
图像已保存: D:\download\gene\ppt_real\ppt\1_29.jpg
帧发生变化: 30, 差分像素1: 0, 差分像素2: 1349
图像已保存: D:\download\gene\ppt_real\ppt\1_30.jpg
帧发生变化: 31, 差分像素1: 0, 差分像素2: 1348
图像已保存: D:\download\gene\ppt_real\ppt\1_31.jpg
帧发生变化: 32, 差分像素1: 3629, 差分像素2: 25817


帧发生变化: 48, 差分像素1: 0, 差分像素2: 8078
图像已保存: D:\download\gene\ppt_real\ppt\3_48.jpg
帧发生变化: 49, 差分像素1: 0, 差分像素2: 7860
图像已保存: D:\download\gene\ppt_real\ppt\3_49.jpg
帧发生变化: 50, 差分像素1: 0, 差分像素2: 3488
图像已保存: D:\download\gene\ppt_real\ppt\3_50.jpg
帧发生变化: 51, 差分像素1: 0, 差分像素2: 3926
图像已保存: D:\download\gene\ppt_real\ppt\3_51.jpg
帧发生变化: 52, 差分像素1: 0, 差分像素2: 4138
图像已保存: D:\download\gene\ppt_real\ppt\3_52.jpg
帧发生变化: 53, 差分像素1: 0, 差分像素2: 6817
图像已保存: D:\download\gene\ppt_real\ppt\3_53.jpg
帧发生变化: 54, 差分像素1: 0, 差分像素2: 7321
图像已保存: D:\download\gene\ppt_real\ppt\3_54.jpg
帧发生变化: 55, 差分像素1: 0, 差分像素2: 11643
图像已保存: D:\download\gene\ppt_real\ppt\3_55.jpg
帧发生变化: 56, 差分像素1: 140, 差分像素2: 2747
图像已保存: D:\download\gene\ppt_real\ppt\3_56.jpg
帧发生变化: 57, 差分像素1: 897, 差分像素2: 38474
图像已保存: D:\download\gene\ppt_real\ppt\3_57.jpg
帧发生变化: 58, 差分像素1: 1338, 差分像素2: 28951
图像已保存: D:\download\gene\ppt_real\ppt\3_58.jpg
帧发生变化: 59, 差分像素1: 2070, 差分像素2: 0
图像已保存: D:\download\gene\ppt_real\ppt\3_59.jpg
帧发生变化: 60, 差分像素1: 1627, 差分像素2: 26163
图像已保存

图像已保存: D:\download\gene\ppt_real\ppt\5_26.jpg
帧发生变化: 27, 差分像素1: 0, 差分像素2: 3669
图像已保存: D:\download\gene\ppt_real\ppt\5_27.jpg
帧发生变化: 28, 差分像素1: 0, 差分像素2: 3530
图像已保存: D:\download\gene\ppt_real\ppt\5_28.jpg
帧发生变化: 29, 差分像素1: 858, 差分像素2: 3980
图像已保存: D:\download\gene\ppt_real\ppt\5_29.jpg
帧发生变化: 30, 差分像素1: 0, 差分像素2: 4034
图像已保存: D:\download\gene\ppt_real\ppt\5_30.jpg
帧发生变化: 31, 差分像素1: 0, 差分像素2: 4235
图像已保存: D:\download\gene\ppt_real\ppt\5_31.jpg
帧发生变化: 32, 差分像素1: 0, 差分像素2: 7634
图像已保存: D:\download\gene\ppt_real\ppt\5_32.jpg
帧发生变化: 33, 差分像素1: 0, 差分像素2: 7917
图像已保存: D:\download\gene\ppt_real\ppt\5_33.jpg
帧发生变化: 34, 差分像素1: 0, 差分像素2: 4126
图像已保存: D:\download\gene\ppt_real\ppt\5_34.jpg
帧发生变化: 35, 差分像素1: 436, 差分像素2: 4207
图像已保存: D:\download\gene\ppt_real\ppt\5_35.jpg
帧发生变化: 36, 差分像素1: 210, 差分像素2: 7415
图像已保存: D:\download\gene\ppt_real\ppt\5_36.jpg
帧发生变化: 37, 差分像素1: 0, 差分像素2: 7504
图像已保存: D:\download\gene\ppt_real\ppt\5_37.jpg
帧发生变化: 38, 差分像素1: 0, 差分像素2: 4043
图像已保存: D:\download\gene\ppt_real\ppt\5_38.jpg


帧发生变化: 14, 差分像素1: 9429, 差分像素2: 16882
图像已保存: D:\download\gene\ppt_real\ppt\4_14.jpg
帧发生变化: 15, 差分像素1: 1387, 差分像素2: 0
图像已保存: D:\download\gene\ppt_real\ppt\4_15.jpg
帧发生变化: 16, 差分像素1: 1387, 差分像素2: 0
图像已保存: D:\download\gene\ppt_real\ppt\4_16.jpg
帧发生变化: 17, 差分像素1: 0, 差分像素2: 8428
图像已保存: D:\download\gene\ppt_real\ppt\4_17.jpg
帧发生变化: 18, 差分像素1: 96, 差分像素2: 10167
图像已保存: D:\download\gene\ppt_real\ppt\4_18.jpg
帧发生变化: 19, 差分像素1: 96, 差分像素2: 12473
图像已保存: D:\download\gene\ppt_real\ppt\4_19.jpg
帧发生变化: 20, 差分像素1: 0, 差分像素2: 7094
图像已保存: D:\download\gene\ppt_real\ppt\4_20.jpg
帧发生变化: 21, 差分像素1: 0, 差分像素2: 6568
图像已保存: D:\download\gene\ppt_real\ppt\4_21.jpg
帧发生变化: 22, 差分像素1: 0, 差分像素2: 11130
图像已保存: D:\download\gene\ppt_real\ppt\4_22.jpg
帧发生变化: 23, 差分像素1: 4741, 差分像素2: 0
图像已保存: D:\download\gene\ppt_real\ppt\4_23.jpg
帧发生变化: 24, 差分像素1: 8064, 差分像素2: 14376
图像已保存: D:\download\gene\ppt_real\ppt\4_24.jpg
帧发生变化: 25, 差分像素1: 3799, 差分像素2: 9293
图像已保存: D:\download\gene\ppt_real\ppt\4_25.jpg
帧发生变化: 26, 差分像素1: 0, 差分像素2: 17595
图像

...

帧发生变化: 33, 差分像素1: 2381, 差分像素2: 6882
图像已保存: D:\download\gene\ppt_real\ppt\24_33.jpg
帧发生变化: 34, 差分像素1: 3182, 差分像素2: 6198
图像已保存: D:\download\gene\ppt_real\ppt\24_34.jpg
帧发生变化: 35, 差分像素1: 598, 差分像素2: 1
图像已保存: D:\download\gene\ppt_real\ppt\24_35.jpg
帧发生变化: 36, 差分像素1: 598, 差分像素2: 1
图像已保存: D:\download\gene\ppt_real\ppt\24_36.jpg
帧发生变化: 37, 差分像素1: 12232, 差分像素2: 10792
图像已保存: D:\download\gene\ppt_real\ppt\24_37.jpg
帧发生变化: 38, 差分像素1: 14437, 差分像素2: 16844
图像已保存: D:\download\gene\ppt_real\ppt\24_38.jpg
帧发生变化: 39, 差分像素1: 5496, 差分像素2: 10815
图像已保存: D:\download\gene\ppt_real\ppt\24_39.jpg
帧发生变化: 40, 差分像素1: 11570, 差分像素2: 19976
图像已保存: D:\download\gene\ppt_real\ppt\24_40.jpg
帧发生变化: 41, 差分像素1: 5023, 差分像素2: 23135
图像已保存: D:\download\gene\ppt_real\ppt\24_41.jpg
帧发生变化: 42, 差分像素1: 3317, 差分像素2: 38839
图像已保存: D:\download\gene\ppt_real\ppt\24_42.jpg
帧发生变化: 43, 差分像素1: 3675, 差分像素2: 39824
图像已保存: D:\download\gene\ppt_real\ppt\24_43.jpg
帧发生变化: 44, 差分像素1: 12196, 差分像素2: 4570
图像已保存: D:\download\gene\ppt_real\ppt\24_44.jpg
帧

图像已保存: D:\download\gene\ppt_real\ppt\25_72.jpg
帧发生变化: 73, 差分像素1: 0, 差分像素2: 16315
图像已保存: D:\download\gene\ppt_real\ppt\25_73.jpg
帧发生变化: 74, 差分像素1: 0, 差分像素2: 9020
图像已保存: D:\download\gene\ppt_real\ppt\25_74.jpg
帧发生变化: 75, 差分像素1: 0, 差分像素2: 5282
图像已保存: D:\download\gene\ppt_real\ppt\25_75.jpg
帧发生变化: 76, 差分像素1: 0, 差分像素2: 1605
图像已保存: D:\download\gene\ppt_real\ppt\25_76.jpg
帧发生变化: 77, 差分像素1: 0, 差分像素2: 1815
图像已保存: D:\download\gene\ppt_real\ppt\25_77.jpg
帧发生变化: 78, 差分像素1: 0, 差分像素2: 36713
图像已保存: D:\download\gene\ppt_real\ppt\25_78.jpg
帧发生变化: 79, 差分像素1: 0, 差分像素2: 38209
图像已保存: D:\download\gene\ppt_real\ppt\25_79.jpg
帧发生变化: 80, 差分像素1: 0, 差分像素2: 15135
图像已保存: D:\download\gene\ppt_real\ppt\25_80.jpg
帧发生变化: 81, 差分像素1: 0, 差分像素2: 40858
图像已保存: D:\download\gene\ppt_real\ppt\25_81.jpg
帧发生变化: 82, 差分像素1: 0, 差分像素2: 13467
图像已保存: D:\download\gene\ppt_real\ppt\25_82.jpg
帧发生变化: 83, 差分像素1: 1319, 差分像素2: 14609
图像已保存: D:\download\gene\ppt_real\ppt\25_83.jpg
帧发生变化: 84, 差分像素1: 3852, 差分像素2: 6957
图像已保存: D:\download\gene\ppt

帧发生变化: 72, 差分像素1: 3064, 差分像素2: 36375
图像已保存: D:\download\gene\ppt_real\ppt\26_72.jpg
帧发生变化: 73, 差分像素1: 1672, 差分像素2: 29945
图像已保存: D:\download\gene\ppt_real\ppt\26_73.jpg
帧发生变化: 74, 差分像素1: 0, 差分像素2: 1845
图像已保存: D:\download\gene\ppt_real\ppt\26_74.jpg
帧发生变化: 75, 差分像素1: 23580, 差分像素2: 30128
图像已保存: D:\download\gene\ppt_real\ppt\26_75.jpg
帧发生变化: 76, 差分像素1: 23580, 差分像素2: 43928
图像已保存: D:\download\gene\ppt_real\ppt\26_76.jpg
帧发生变化: 77, 差分像素1: 10244, 差分像素2: 39891
图像已保存: D:\download\gene\ppt_real\ppt\26_77.jpg
帧发生变化: 78, 差分像素1: 0, 差分像素2: 7865
图像已保存: D:\download\gene\ppt_real\ppt\26_78.jpg
帧发生变化: 79, 差分像素1: 0, 差分像素2: 8286
图像已保存: D:\download\gene\ppt_real\ppt\26_79.jpg
帧发生变化: 80, 差分像素1: 0, 差分像素2: 3866
图像已保存: D:\download\gene\ppt_real\ppt\26_80.jpg
帧发生变化: 81, 差分像素1: 0, 差分像素2: 3641
图像已保存: D:\download\gene\ppt_real\ppt\26_81.jpg
帧发生变化: 82, 差分像素1: 0, 差分像素2: 5762
图像已保存: D:\download\gene\ppt_real\ppt\26_82.jpg
帧发生变化: 83, 差分像素1: 0, 差分像素2: 5754
图像已保存: D:\download\gene\ppt_real\ppt\26_83.jpg
帧发生变化: 84, 差分像素1:

帧发生变化: 82, 差分像素1: 0, 差分像素2: 37352
图像已保存: D:\download\gene\ppt_real\ppt\27_82.jpg
帧发生变化: 83, 差分像素1: 0, 差分像素2: 39614
图像已保存: D:\download\gene\ppt_real\ppt\27_83.jpg
帧发生变化: 84, 差分像素1: 0, 差分像素2: 13531
图像已保存: D:\download\gene\ppt_real\ppt\27_84.jpg
帧发生变化: 85, 差分像素1: 32, 差分像素2: 11772
图像已保存: D:\download\gene\ppt_real\ppt\27_85.jpg
帧发生变化: 86, 差分像素1: 32, 差分像素2: 11554
图像已保存: D:\download\gene\ppt_real\ppt\27_86.jpg
帧发生变化: 87, 差分像素1: 9898, 差分像素2: 37519
图像已保存: D:\download\gene\ppt_real\ppt\27_87.jpg
帧发生变化: 88, 差分像素1: 25128, 差分像素2: 15152
图像已保存: D:\download\gene\ppt_real\ppt\27_88.jpg
帧发生变化: 89, 差分像素1: 25178, 差分像素2: 38638
图像已保存: D:\download\gene\ppt_real\ppt\27_89.jpg
帧发生变化: 90, 差分像素1: 1704, 差分像素2: 22708
图像已保存: D:\download\gene\ppt_real\ppt\27_90.jpg
帧发生变化: 91, 差分像素1: 1779, 差分像素2: 0
图像已保存: D:\download\gene\ppt_real\ppt\27_91.jpg
帧发生变化: 92, 差分像素1: 3219, 差分像素2: 22709
图像已保存: D:\download\gene\ppt_real\ppt\27_92.jpg
帧发生变化: 93, 差分像素1: 36, 差分像素2: 15019
图像已保存: D:\download\gene\ppt_real\ppt\27_93.jpg
帧发生变化: 94

帧发生变化: 46, 差分像素1: 4356, 差分像素2: 16570
图像已保存: D:\download\gene\ppt_real\ppt\28_46.jpg
帧发生变化: 47, 差分像素1: 2537, 差分像素2: 15350
图像已保存: D:\download\gene\ppt_real\ppt\28_47.jpg
帧发生变化: 48, 差分像素1: 0, 差分像素2: 8002
图像已保存: D:\download\gene\ppt_real\ppt\28_48.jpg
帧发生变化: 49, 差分像素1: 0, 差分像素2: 3069
图像已保存: D:\download\gene\ppt_real\ppt\28_49.jpg
帧发生变化: 50, 差分像素1: 1067, 差分像素2: 19017
图像已保存: D:\download\gene\ppt_real\ppt\28_50.jpg
帧发生变化: 51, 差分像素1: 1064, 差分像素2: 17118
图像已保存: D:\download\gene\ppt_real\ppt\28_51.jpg
帧发生变化: 52, 差分像素1: 0, 差分像素2: 17099
图像已保存: D:\download\gene\ppt_real\ppt\28_52.jpg
帧发生变化: 53, 差分像素1: 1037, 差分像素2: 20151
图像已保存: D:\download\gene\ppt_real\ppt\28_53.jpg
帧发生变化: 54, 差分像素1: 909, 差分像素2: 20059
图像已保存: D:\download\gene\ppt_real\ppt\28_54.jpg
帧发生变化: 55, 差分像素1: 0, 差分像素2: 1170
图像已保存: D:\download\gene\ppt_real\ppt\28_55.jpg
帧发生变化: 56, 差分像素1: 152, 差分像素2: 0
图像已保存: D:\download\gene\ppt_real\ppt\28_56.jpg
帧发生变化: 57, 差分像素1: 1214, 差分像素2: 19720
图像已保存: D:\download\gene\ppt_real\ppt\28_57.jpg
帧发生变化: 58, 差分